In [8]:
import os
import torch
import json

os.environ["MKL_NUM_THREADS"]="1"
os.environ["NUMEXPR_NUM_THREADS"]="1"
os.environ["OMP_NUM_THREADS"]="1"
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6,7"

import sys

import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import random_split
from torch_geometric.data import DataLoader

from spatial.merfish_dataset import FilteredMerfishDataset, MerfishDataset, SyntheticNonlinear, SyntheticDataset0, SyntheticDataset1, SyntheticDataset2, SyntheticDataset3
from spatial.models.monet_ae import MonetAutoencoder2D, TrivialAutoencoder, MonetDense
from spatial.train import train
from spatial.predict import test

import warnings
warnings.filterwarnings("ignore")

In [14]:
import hydra
from hydra.experimental import compose, initialize

test_loss_rad_dict = {}

for rad in [15]:
    with initialize(config_path="../config"):
        cfg_from_terminal = compose(config_name="config")
        OmegaConf.update(cfg_from_terminal, "model.kwargs.observables_dimension", 71)
        OmegaConf.update(cfg_from_terminal, "model.kwargs.hidden_dimensions", [512, 512, 512, 512, 512, 512])
        OmegaConf.update(cfg_from_terminal, "model.kwargs.output_dimension", 84)
        OmegaConf.update(cfg_from_terminal, "optimizer.name", "Adam")
        OmegaConf.update(cfg_from_terminal, "training.logger_name", "KFOLD_test")
        OmegaConf.update(cfg_from_terminal, "datasets.dataset.include_celltypes", False)
        OmegaConf.update(cfg_from_terminal, "model.kwargs.include_skip_connections", True)
        OmegaConf.update(cfg_from_terminal, "radius", rad)
        OmegaConf.update(cfg_from_terminal, "training.trainer.strategy", "auto")
        # OmegaConf.update(cfg_from_terminal, "gpus", [2])
        OmegaConf.update(cfg_from_terminal, "training.filepath", cfg_from_terminal.training.filepath)
        print(cfg_from_terminal.training.filepath)
        output = test(cfg_from_terminal)
        trainer, l1_losses, inputs, gene_expressions, celltypes, test_results = output
        test_loss_rad_dict[rad] = test_results[0]['test_loss']

MonetDense__[512, 512, 512, 512, 512, 512]__15__KFOLD_test__Adam


100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:20<00:00,  1.16it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [5,6,7]


Testing: |                                                                                               | 0/?…

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.4102255403995514     │
│       test_loss_mse       │    0.4102255403995514     │
└───────────────────────────┴───────────────────────────┘

TEST Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                         	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                          	|  -              	|  366            	